In [1]:
import torch

print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
print("Is CUDA available?:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))


ImportError: /home/vikhil/miniconda3/envs/vikhil/lib/python3.10/site-packages/torch/lib/libtorch_cpu.so: undefined symbol: iJIT_NotifyEvent

In [ ]:
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from torchtext.data.utils import get_tokenizer
#from torchtext.vocab import build_vocab_from_iterator
#from torch.nn.utils.rnn import pad_sequence
import torch
import nltk
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torchtext.transforms import BasicEnglishNormalize, Tokenizer

# Example tokenizer
from torchtext.data.utils import get_tokenizer  # For older versions use below

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

class GPUTextPreprocessor:
    def __init__(self, texts, device='cuda', perform_spell_correction=False):
        self.device = device if torch.cuda.is_available() else 'cpu'
        self.tokenizer = get_tokenizer("basic_english")
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.perform_spell_correction = perform_spell_correction
        
        if self.perform_spell_correction:
            from spellchecker import SpellChecker
            self.spell = SpellChecker()

        self.cleaned_texts = [self.clean_text(t) for t in texts]
        def token_iterator():
            for text in self.cleaned_texts:
                yield self.tokenizer(text)
        self.vocab = build_vocab_from_iterator(token_iterator(), specials=["<unk>"])
        self.vocab.set_default_index(self.vocab["<unk>"])

    def clean_text(self, text):
        if pd.isna(text): return ""
        text = BeautifulSoup(text, "html.parser").get_text()
        text = text.lower().translate(str.maketrans('', '', string.punctuation))
        tokens = [t for t in self.tokenizer(text) if t not in self.stop_words and t.isalpha()]
        if self.perform_spell_correction:
            tokens = [self.spell.correction(t) for t in tokens]
        tokens = [self.lemmatizer.lemmatize(t) for t in tokens]
        return ' '.join(tokens)

    def encode_tensor(self, cleaned_text):
        tokens = self.tokenizer(cleaned_text)
        indices = [self.vocab[token] for token in tokens]
        return torch.tensor(indices, dtype=torch.long).to(self.device)

    def batch_encode(self):
        tensors = [self.encode_tensor(t) for t in self.cleaned_texts if t]
        if not tensors: return torch.tensor([]).to(self.device)
        return pad_sequence(tensors, batch_first=True, padding_value=self.vocab["<unk>"])
